In [1]:
#this code reads projects from master file, combines data from RTS
#this code downloads ESAs from section 52 applications

import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
import wget
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Folder analysis on PDFs and CSVs

In [2]:
csv_path = 'F:/Environmental Baseline Data/Version 2/Data/CSV'  
csv_paths = os.listdir(csv_path)
list_csvs_full_path = [csv_path +  '/' + str(x) for x in csv_paths]

pdf_path = 'F:/Environmental Baseline Data/Version 2/Data/PDFs'  
pdf_paths = os.listdir(pdf_path)
list_pdfs_full_path = [pdf_path +  '/' + str(x) for x in pdf_paths]

# len(all_paths_folder)len(all_paths_folder)

In [3]:
#number of files in each folder
len(list_pdfs_full_path)
len(list_csvs_full_path)


1322

25077

In [4]:
#extract postal from pdfs

pdfs_df = pd.DataFrame(list_pdfs_full_path)
pdfs_df.columns = ['pdf path']
postal_series = pdfs_df['pdf path'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
pdfs_df.insert(1, 'postal', postal_series)

#check for nulls
pdfs_df['postal'].isnull().sum()
pdfs_df.loc[pdfs_df['postal'].isnull()] 


2

,pdf path,postal
1320,F:/Environmental Baseline Data/Version 2/Data/...,NaN
1321,F:/Environmental Baseline Data/Version 2/Data/...,NaN


In [5]:
#extract postal from csvs
csvs_df = pd.DataFrame(list_csvs_full_path)
csvs_df.columns = ['csv path']
postal_series = csvs_df['csv path'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
csvs_df.insert(1, 'postal', postal_series)
csvs_df['postal'].isnull().sum()

csvs_df['postal'].isnull().sum()
nulls = csvs_df.loc[csvs_df['postal'].isnull()] 

0

0

In [6]:
# csvs extracted from documents; 
len(csvs_df['postal'])
csvs_df['postal'].nunique()
csvs_df['postal upper'] = csvs_df['postal'].str.upper()

len(pdfs_df['postal'])
pdfs_df['postal'].nunique()
pdfs_df['postal upper'] = pdfs_df['postal'].str.upper()

csvs_df['postal'].nunique()/pdfs_df['postal'].nunique()

25077

796

1322

1317

0.6044039483675019

## Create Index 2

In [7]:
df3 = csvs_df.merge(pdfs_df, on = 'postal upper')
df3.head()
df3.shape
df3.to_csv('test1.csv')
df3['postal upper'].isnull().sum()
nulls_df3 = df3.loc[df3['postal upper'].isnull()]  

,csv path,postal_x,postal upper,pdf path,postal_y
0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0
1,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0
2,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0
3,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0
4,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0


(25154, 5)

0

In [8]:
df3['postal upper'].nunique()
len(df3['postal upper'])

796

25154

In [9]:
#identify pdfs with no tables
total_pdfs_list = list(pdfs_df['postal upper'].unique())
total_csv_list = list(df3['postal upper'].unique())
pdfs_no_tables = list(set(total_pdfs_list) - set(total_csv_list))
pdfs_no_tables_df = pd.DataFrame(pdfs_no_tables)
pdfs_no_tables_df.shape
pdfs_no_tables_df.to_csv('PDFs with no tables.csv')
#

(522, 1)

## Add Project Name to Index 2

In [10]:
#list of ESAs by project

master = pd.read_csv('F:/Environmental Baseline Data/Version 2/Support files/list_of_Section_52_projects.csv', encoding='ISO-8859-1')
master.shape
master['Application title'].nunique()

#convert links
urls_list = master['link to folder with ESA'].tolist()
urls_list_load = [str(x).replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/View/', 
                                 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/') for x in urls_list]

all_esa = []

for x in urls_list_load:
    cookies = {'RDI-NumberOfRecords': '200'}
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'lxml')
    tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
    file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
    hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
    df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'folder' : str(x)})
    all_esa.append(df)
    
df_esa = pd.concat(all_esa, axis = 0)
df_esa = df_esa[df_esa['file_name'].str.contains('ESA|nvironment|Horn|ocio|EIA|App 13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro')]
df_esa['Folder link'] = df_esa['folder'].str.replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/', '').astype(int)
df_esa['file_name'].nunique()
df_esa.head()

#add ESA info to master
master_esa = df_esa.merge(master, on = 'Folder link')
postal_series = master_esa['file_name'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
master_esa.insert(1, 'postal', postal_series)
master_esa['postal upper'] = master_esa['postal'].str.upper()

#regular expressions to parse out application date and hearing order 
application_date = master_esa['Application title'].str.extract('(\d\d\d\d-\d\d-\d\d)')
master_esa.insert(2, 'Application filing date', application_date)

hearing_order = master_esa['Application title'].str.extract(r'(\D\D-\d(.+?)\d\d\d\d)')
hearing_order = hearing_order.drop([1], axis=1)
master_esa.insert(4, 'Hearing order', hearing_order)

#clean up 'Application title' column to new column 'Application title short'
master_esa['Application title short'] = master_esa['Application title'].str.replace(r'(\d\d\d\d-\d\d-\d\d)', '', regex = True)
master_esa['Application title short'] = master_esa['Application title short'].str.replace(r'(\D\D-\d(.+?)\d\d\d\d)', '', regex = True)
master_esa['Application title short'] = master_esa['Application title short'].str.replace("(", '').str.replace(")", '').str.replace(' - ', '').str.replace(' \x96 ', '').str.strip()

#clean up download link
master_esa['hrefs'] = master_esa['hrefs'].str.replace('/REGDOCS', 'https://apps.cer-rec.gc.ca/REGDOCS')
master_esa = master_esa.rename(columns={'href' : 'download link'})
master_esa['file_name'].nunique()

(67, 6)

36

1342

,file_name,hrefs,folder,Folder link
5,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,/REGDOCS/File/Download/268706,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693
6,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,/REGDOCS/File/Download/268709,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693
7,A0H8C2 - 13.0 EIA - Section 13.7 Wildlife Part 2,/REGDOCS/File/Download/268712,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693
8,A0H8C3 - 13.0 EIA - Section 13.8 to 13.13,/REGDOCS/File/Download/269018,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693
9,A0H8C4 - 13.1 App 13A - Alignment Sheets,/REGDOCS/File/Download/269021,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693


1342

In [11]:
#merge with csvs
df_csv_with_project = df3.merge(master_esa, on = 'postal upper')
df_csv_with_project['postal upper'].nunique()

796

In [12]:
df_csv_with_project.shape

(25308, 18)

In [13]:
df_csv_with_project.columns

Index(['csv path', 'postal_x', 'postal upper', 'pdf path', 'postal_y',
       'file_name', 'postal', 'Application filing date', 'hrefs',
       'Hearing order', 'folder', 'Folder link', 'Section',
       'Application title', 'link to folder with ESA', 'Company Name',
       'Consultant Name', 'Application title short'],
      dtype='object')

## Add GIS attributes to Index 2

In [580]:
#read data provided by Tammy
df_rts = pd.read_csv('F:\Environmental Baseline Data\Version 2\Support files\List of Section 52 Projects - RTS details from Tammy.csv', encoding='ISO-8859-1')
df_rts = df_rts.drop_duplicates(subset = 'Application title')
df_rts = df_rts.ffill(axis = 0)
df_rts.shape
df_rts.columns
df_rts['Application title'].nunique()

# master.shape
# master.columns
# master['postal'].nunique()

(36, 9)

Index(['Section', 'Application title', 'link to folder with ESA',
       'Folder link', 'Company Name', 'Status in GIS',
       'RTS English Title = Project name', 'Regulatory Instruments Issued',
       'Pipeline Named in GIS'],
      dtype='object')

36

In [581]:
csvs_with_gis = df_csv_with_project.merge(df_rts[['Application title', 'Status in GIS', 'RTS English Title = Project name', 'Regulatory Instruments Issued', 'Pipeline Named in GIS']], on = 'Application title')

In [606]:
csvs_with_gis.shape
csvs_with_gis.columns
csvs_with_gis.head(2)

(25308, 22)

Index(['csv path', 'postal_x', 'postal upper', 'pdf path', 'postal_y',
       'file_name', 'postal', 'Application filing date', 'hrefs',
       'Hearing order', 'folder', 'Folder link', 'Section',
       'Application title', 'link to folder with ESA', 'Company Name',
       'Consultant Name', 'Application title short', 'Status in GIS',
       'RTS English Title = Project name', 'Regulatory Instruments Issued',
       'Pipeline Named in GIS'],
      dtype='object')

,csv path,postal_x,postal upper,pdf path,postal_y,file_name,postal,Application filing date,hrefs,Hearing order,...,Section,Application title,link to folder with ESA,Company Name,Consultant Name,Application title short,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS
0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,A0H8C0,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,...,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline
1,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,A0H8C0,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,...,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline


## Add CSV metadata to Index 2

In [716]:
path = 'F:/Environmental Baseline Data/Version 2/Data/CSV'  
csv_paths = os.listdir(path)
all_paths_folder = [path +  '/' + str(x) for x in csv_paths]
len(all_paths_folder)

25077

In [634]:
#go through each full path and grab the headers, shape and size of file; combine into a df

# headers = []
# path_file = []
# columns_count = []
# rows = []
# missing = []
# file_size = []

# for x in all_paths_folder: #limit scope via list
#     try:
#         df = pd.read_csv(x)
#         path_file.append(x)
#         headers.append(df.columns.to_list())
#         columns_count.append(len(df.columns))
#         rows.append(len(df.index))
#         file_size.append(os.path.getsize(x))
#     except:
#         missing.append(x)


In [635]:
#test number of csv that cannot be accessed 
len(missing)
len(file_size)

25

25052

In [746]:
csv_temp = pd.DataFrame({'csv path': path_file, 'headers': headers, 'number of columns' : columns_count, 
                   'number of rows' : rows, 'file size' : file_size})

postal_series_csv = csv_temp['csv path'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
csv_temp.insert(1, 'postal', postal_series_csv)
csv_temp['postal upper'] = csv_temp['postal'].str.upper()
# csv_temp['headers'] = csv_temp['headers'].str.replace('[', '')
csv_temp = csv_temp.drop_duplicates(subset = 'csv path')
csv_temp.shape #some csv files were not read
csv_temp['postal upper'].nunique()

(25052, 7)

795

In [747]:
csvs_with_gis_metadata = csv_temp.merge(csvs_with_gis, on = 'csv path')
csvs_with_gis_metadata = csvs_with_gis_metadata.drop_duplicates(subset = 'csv path')
csvs_with_gis_metadata.shape

(25052, 28)

In [748]:
#cleanup headers component
csvs_with_gis_metadata['headers'] = csvs_with_gis_metadata['headers']
csvs_with_gis_metadata['headers'] = csvs_with_gis_metadata['headers'].apply(', '.join)

# csvs_with_gis_metadata['headers'] = [','.join(map(str, l)) for l in csvs_with_gis_metadata['headers']] #convert list to string for each row

#future additions 
csvs_with_gis_metadata['ESA data category'] = 'TBD'
csvs_with_gis_metadata['Figure title'] = 'TBD'
csvs_with_gis_metadata['Geotag location'] = 'TBD'

In [751]:
csvs_with_gis_metadata.to_csv('F:\Environmental Baseline Data\Version 2\Beta deliverables\Index 2 - CSVs - V1.csv')
csvs_with_gis_metadata.head()

,csv path,postal_x,headers,number of columns,number of rows,file size,postal upper_x,postal_x,postal upper_y,pdf path,...,Company Name,Consultant Name,Application title short,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,ESA data category,Figure title,Geotag location
0,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,"Assessment Scenario, Issue, Magnitude/Extent1,...",6,6,822,A0H8C0,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,TBD,TBD,TBD
1,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,"Site Series, Hypothetical Baseline, Unnamed: 2...",14,11,1585,A0H8C0,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,TBD,TBD,TBD
2,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,"Structural Stage, Hypothetical Baseline, Unnam...",14,10,1251,A0H8C0,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,TBD,TBD,TBD
3,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,"Assessment Scenario, Issue/ Measurable Paramet...",6,5,482,A0H8C0,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,TBD,TBD,TBD
4,F:/Environmental Baseline Data/Version 2/Data/...,A0H8C0,"Assessment Scenario, Issue/Measurable Paramete...",6,6,708,A0H8C0,A0H8C0,A0H8C0,F:/Environmental Baseline Data/Version 2/Data/...,...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Application to Construct and Operate Ekwan Pip...,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,TBD,TBD,TBD


## Create Index 1 (36 projects ∴ 36-rows Index)

In [695]:
master = pd.read_csv('F:/Environmental Baseline Data/Version 2/Support files/list_of_Section_52_projects.csv', encoding='ISO-8859-1')
master.shape
master['Application title'].nunique()
master.head()

(67, 6)

36

,Section,Application title,link to folder with ESA,Folder link,Company Name,Consultant Name
0,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,268693,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.
1,52,2006-05-23 - Application for the Brunswick Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,408713,Emera Brunswick Pipeline Company Ltd.,Jacques Whitford
2,52,"2007-03-09 - Application for Line 13 Transfer,...",https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,456315,Enbridge Southern Lights GP Inc.,TERA Environmental Consultants
3,52,2007-06-28 - Application for the Line 4 Extens...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,467952,Enbridge Pipelines Inc.,TERA Environmental Consultants
4,52,2008-02-27 - Application to construct and oper...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/5...,500494,Westcoast Energy Inc.,TERA Environmental Consultants


In [715]:
#merge with RTS data
index1_with_gis = master.merge(df_rts[['Application title', 'Status in GIS', 'RTS English Title = Project name', 'Regulatory Instruments Issued', 'Pipeline Named in GIS']], on = 'Application title')

#regular expressions to parse out application date and hearing order 
application_date = index1_with_gis['Application title'].str.extract('(\d\d\d\d-\d\d-\d\d)')
index1_with_gis.insert(2, 'Application filing date', application_date)

#regular expressions to parse out hearing order 
hearing_order = index1_with_gis['Application title'].str.extract(r'(\D\D-\d(.+?)\d\d\d\d)')
hearing_order = hearing_order.drop([1], axis=1)
index1_with_gis.insert(4, 'Hearing order', hearing_order)


#clean up 'Application title' column to new column 'Application title short'
index1_with_gis['Application title short'] = index1_with_gis['Application title'].str.replace(r'(\d\d\d\d-\d\d-\d\d)', '', regex = True)
index1_with_gis['Application title short'] = index1_with_gis['Application title short'].str.replace(r'(\D\D-\d(.+?)\d\d\d\d)', '', regex = True)
index1_with_gis['Application title short'] = index1_with_gis['Application title short'].str.replace("(", '').str.replace(")", '').str.replace(' - ', '').str.replace(' \x96 ', '').str.strip()
index1_with_gis = index1_with_gis.drop_duplicates(subset = 'Application title')

#add link to Application
#add link to Decision
index1_with_gis['Link to Application'] = 'TBD'
index1_with_gis['Link to Decision'] = 'TBD'
index1_with_gis['Location'] = 'TBD'

index1_with_gis.shape
index1_with_gis.head()
index1_with_gis.to_csv('F:\Environmental Baseline Data\Version 2\Beta deliverables\Index 1 - Projects - V1.csv')

(36, 16)

,Section,Application title,Application filing date,link to folder with ESA,Hearing order,Folder link,Company Name,Consultant Name,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location
0,52,2003-03-17 Application to Construct and Operat...,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,GH-1-2003,268693,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,Part provincially regulated,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,Application to Construct and Operate Ekwan Pip...,TBD,TBD,TBD
1,52,2006-05-23 - Application for the Brunswick Pip...,2006-05-23,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,GH-1-2006,408713,Emera Brunswick Pipeline Company Ltd.,Jacques Whitford,Operating,Brunswick Pipeline Project,GC-110,Brunswick Pipeline,Application for the Brunswick Pipeline Project,TBD,TBD,TBD
2,52,"2007-03-09 - Application for Line 13 Transfer,...",2007-03-09,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,OH-3-2007,456315,Enbridge Southern Lights GP Inc.,TERA Environmental Consultants,Operating,Line 13 Reversal Facilities Project,XO-E242-03-2008,Line 13,"Application for Line 13 Transfer, Line 13 Reve...",TBD,TBD,TBD
3,52,2007-06-28 - Application for the Line 4 Extens...,2007-06-28,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,OH-5-2007,467952,Enbridge Pipelines Inc.,TERA Environmental Consultants,Operating,Line 4 Extension Project,OC-55,Line 4,Application for the Line 4 Extension Project,TBD,TBD,TBD
4,52,2008-02-27 - Application to construct and oper...,2008-02-27,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/5...,GH-3-2008,500494,Westcoast Energy Inc.,TERA Environmental Consultants,Operating,South Peace Pipeline Project,GC-112,South Peace Pipeline,Application to construct and operate the South...,TBD,TBD,TBD


## Create Index 3

In [756]:
#list of ESAs by project

master = pd.read_csv('F:/Environmental Baseline Data/Version 2/Support files/list_of_Section_52_projects.csv', encoding='ISO-8859-1')
urls_list = master['link to folder with ESA'].tolist()
urls_list_load = [str(x).replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/View/', 
                                 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/') for x in urls_list]

# all_esa = []

# for x in urls_list_load:
#     cookies = {'RDI-NumberOfRecords': '200'}
#     r = requests.get(x, cookies = cookies)
#     soup = bs(r.text, 'lxml')
#     tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
#     file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
#     hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
#     df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'folder' : str(x)})
#     all_esa.append(df)
    
df_esa = pd.concat(all_esa, axis = 0)
df_esa = df_esa[df_esa['file_name'].str.contains('ESA|nvironment|Horn|ocio|EIA|App 13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro')]
df_esa['Folder link'] = df_esa['folder'].str.replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/', '').astype(int)

#add ESA info to master
master_esa = df_esa.merge(master, on = 'Folder link')
postal_series = master_esa['file_name'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
master_esa.insert(1, 'postal', postal_series)
master_esa['postal upper'] = master_esa['postal'].str.upper()

#regular expressions to parse out application date and hearing order 
application_date = master_esa['Application title'].str.extract('(\d\d\d\d-\d\d-\d\d)')
master_esa.insert(2, 'Application filing date', application_date)

#regular expressions to parse out hearing order 
hearing_order = master_esa['Application title'].str.extract(r'(\D\D-\d(.+?)\d\d\d\d)')
hearing_order = hearing_order.drop([1], axis=1)
master_esa.insert(4, 'Hearing order', hearing_order)

#clean up 'Application title' column to new column 'Application title short'
master_esa['Application title short'] = master_esa['Application title'].str.replace(r'(\d\d\d\d-\d\d-\d\d)', '', regex = True)
master_esa['Application title short'] = master_esa['Application title short'].str.replace(r'(\D\D-\d(.+?)\d\d\d\d)', '', regex = True)
master_esa['Application title short'] = master_esa['Application title short'].str.replace("(", '').str.replace(")", '').str.replace(' - ', '').str.replace(' \x96 ', '').str.strip()

#clean up download link
master_esa['hrefs'] = master_esa['hrefs'].str.replace('/REGDOCS', 'https://apps.cer-rec.gc.ca/REGDOCS')
master_esa = master_esa.rename(columns={'hrefs' : 'download link'})

#future content
master_esa['file size'] = 'TBD'
master_esa['page count'] = 'TBD'

#save to csv
master_esa.to_csv('F:\Environmental Baseline Data\Version 2\Beta deliverables\Index 3 - PDFs - V1.csv')

In [755]:
master_esa.shape
master_esa.head()

(1345, 16)

,file_name,postal,Application filing date,download link,Hearing order,folder,Folder link,Section,Application title,link to folder with ESA,Company Name,Consultant Name,postal upper,Application title short,file size,page count
0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,A0H8C0,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C0,Application to Construct and Operate Ekwan Pip...,TBD,TBD
1,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,A0H8C1,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C1,Application to Construct and Operate Ekwan Pip...,TBD,TBD
2,A0H8C2 - 13.0 EIA - Section 13.7 Wildlife Part 2,A0H8C2,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C2,Application to Construct and Operate Ekwan Pip...,TBD,TBD
3,A0H8C3 - 13.0 EIA - Section 13.8 to 13.13,A0H8C3,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C3,Application to Construct and Operate Ekwan Pip...,TBD,TBD
4,A0H8C4 - 13.1 App 13A - Alignment Sheets,A0H8C4,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C4,Application to Construct and Operate Ekwan Pip...,TBD,TBD
